In [4]:
import pandas as pd
import numpy as np
import json

In [5]:
def search(myDict, lookup):
    matching_stns = {}
    for key, value in myDict.items():
        if lookup in value:
             matching_stns[key]=value
    if matching_stns == []:
        raise ValueError('None of the available stations matches your choice')
    return matching_stns

In [6]:
wrk_dir = "/Users/daniel/Desktop/hourly_aemet/"
with open(f'{wrk_dir}avail_stations.json', 'r') as f:
    locs = json.load(f)

In [7]:
stn_name = 'Barcelona'

In [8]:
matching_stns = search(locs,stn_name.upper())
matching_stns_lc = search(locs,stn_name)

In [ ]:
db_list = list(matching_stns.values())+list(matching_stns_lc.values())
print("Available Databases:")
for i, db_name in enumerate(db_list, start=1):
    print ('{}. {}'.format(i, db_name))


while True:
    try:
        selected = int(input('Select a database (1-{}): '.format(i)))
        db_name = db_list[selected-1]
        print('You have selected {}'.format(db_name))
        stn_id = list(locs.keys())[list(locs.values()).index(db_name)]
        break
    except (ValueError, IndexError):
        print('This is not a valid selection. Please enter number between 1 and {}!'.format(i))




Available Databases:
1. BARCELONA/AEROPUERTO
2. BARCELONA CMT
3. BARCELONA  DRASSANES


In [ ]:
{stn_id}

In [16]:
ifile = f"{wrk_dir}hrly_obs/{stn_id}.txt"
obs = pd.read_csv(ifile, index_col='datetime',parse_dates=True, sep=',',header=0, skiprows=5)

In [20]:
date_index = pd.date_range('2021-03-01','2022-03-15',freq='1H')
obs_new = pd.DataFrame(np.ones((len(date_index),len(obs.columns)))*np.nan,columns=obs.columns)
obs_new['datetime']=[date_index[i] for i in range(len(date_index))]
obs_new = obs_new.set_index('datetime',drop=True)
for nt,ntime in enumerate(obs.index):
    for f,field in enumerate(obs_new.columns):
        if field in obs.columns:
            obs_new.loc[obs_new.index==ntime,field]=obs.loc[obs.index==ntime,field].values

In [21]:
prec_period  = obs_new.prec

In [33]:
pr_day=prec_period.resample("D").sum()


In [36]:
pr_day.where(pr_day>20).dropna()

datetime
2021-11-10    27.0
2021-11-11    36.6
2021-11-23    40.6
2022-03-12    22.6
Name: prec, dtype: float64

In [39]:
prec_period.loc['2021-11-10']

datetime
2021-11-10 00:00:00    2.4
2021-11-10 01:00:00    0.0
2021-11-10 02:00:00    2.8
2021-11-10 03:00:00    2.8
2021-11-10 04:00:00    0.0
2021-11-10 05:00:00    0.0
2021-11-10 06:00:00    0.0
2021-11-10 07:00:00    0.0
2021-11-10 08:00:00    0.0
2021-11-10 09:00:00    0.0
2021-11-10 10:00:00    7.8
2021-11-10 11:00:00    2.2
2021-11-10 12:00:00    0.6
2021-11-10 13:00:00    4.6
2021-11-10 14:00:00    2.2
2021-11-10 15:00:00    0.2
2021-11-10 16:00:00    0.0
2021-11-10 17:00:00    0.2
2021-11-10 18:00:00    0.0
2021-11-10 19:00:00    0.0
2021-11-10 20:00:00    0.0
2021-11-10 21:00:00    0.4
2021-11-10 22:00:00    0.2
2021-11-10 23:00:00    0.6
Name: prec, dtype: float64

In [3]:
srain = pd.Series(obs)

srain =srain.to_frame()
srain = srain.rename(columns = {0:'pr'})

srain['event_start'] = (srain['pr'].astype(bool).shift() != srain['pr'].astype(bool))
srain['event_end'] = (srain['pr'].astype(bool).shift(-1) != srain['pr'].astype(bool))
srain['event'] = srain['event_start'].cumsum()

wet_event_intensity = srain.groupby('event')['pr'].sum()
wet_event_duration = srain.groupby('event')['pr'].count()
srain['event_pr'] = srain['event'].map(wet_event_intensity).where(srain['event_end'])
srain['event_dur'] =  srain['event'].map(wet_event_duration).where(srain['event_end'])

NameError: name 'obs' is not defined